tensor([[ 0.0004],
        [-0.0110]], grad_fn=<AddmmBackward0>)

# 5.2 参数管理

In [16]:
import numpy as np
import pandas as pd
import torch
import os
os.environ ["KMP_DUPLICATE_LIB_OK"] ="TRUE"
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0561],
        [0.0413]], grad_fn=<AddmmBackward0>)

## 1. 参数访问
可以通过net[idx]索引的方式来访问模型的任意层

In [17]:
print(net[2].state_dict()) # 第二层的参数

OrderedDict([('weight', tensor([[ 0.1609,  0.0495, -0.0943,  0.3514,  0.2920,  0.1390,  0.1987, -0.2542]])), ('bias', tensor([0.1077]))])


### 访问具体的参数 如w、bais等

In [18]:
# 访问底层数值
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)


<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1077], requires_grad=True)
tensor([0.1077])


In [19]:
net[2].weight.grad == None

True

### 一次性访问所有参数

In [20]:

print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [21]:
# 另一种访问网络参数的方式
net.state_dict()['2.bias'].data

tensor([0.1077])

### 嵌套块收集参数
首先定义一个生成块的函数，然后在大块当中调用生成一些小块

In [22]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net


rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[-0.3058],
        [-0.3057]], grad_fn=<AddmmBackward0>)

In [23]:
# rgnet的结构
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 5.2 参数的初始化
可以使用内置的初始化函数init.normal_()、init.zeros_等，也可以自定义

In [24]:
#访问第一个主要块中、第二个子块第一层的偏置项
rgnet[0][1][0].bias.data

tensor([-0.2990, -0.1957, -0.3816, -0.4557,  0.4207,  0.3273, -0.3753, -0.4769])

### 5.2.1 内置初始化参数

In [25]:
# 内置初始化器
def init_normal(m):
    if type(m) == nn.Linear:
        # 权重初始化为（0，0.01）的高斯分布
        nn.init.normal_(m.weight, mean = 0, std = 0.01)
        # 偏置全部初始化为0
        nn.init.zeros_(m.bias)
    

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]



(tensor([ 0.0252,  0.0094,  0.0005, -0.0049]), tensor(0.))

In [26]:
# 参数初始化为给定的常数1
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [27]:
def init_xavier(m):
    if type(m) == nn.Linear:
        #将输入的权重张量用均匀分布进行初始化
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

# 不同层不同的初始化参数的方法
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.0425,  0.4593,  0.3631, -0.0651])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 5.2.2 自定义初始化

In [28]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 绝对值小于5的进行减枝
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -0.0000,  0.0000, -0.0000],
        [-5.5983, -6.3845, -0.0000,  8.8732]], grad_fn=<SliceBackward0>)

### 5.2.3参数绑定
首先给一组参数取一个名称，然后在后续组合当中使用它
所有使用该名称的层都是这一组参数

In [29]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
